# Example function to standardize experiment folder structure

In [ ]:
import os
import shutil

def restructure_experiment_folders(base_dir):
    """Restructures a nested experiment folder hierarchy into a standardized format.

    This function recursively traverses `base_dir`, identifies folders containing image files,
    and moves them into a new folder structure organized by:
    `Stim-{stim_type}_exp/{subject}/{state}/{hemisphere}-{stim_type}`.

    The expected original directory structure must contain at least 7 path levels, with the last
    seven expected to be: year, pre-experiment folder, state, stimulation type, hemisphere, and subject.

    Args:
        base_dir (str or Path): Root directory containing the original experiment folders.

    Example:
        Given a directory like:
            base_dir/2023/before_exp/Awake/HL/L/mouse123
        The images will be moved to:
            base_dir/Stim-HL_exp/mouse123/Awake/L-HL

    """
    for root, dirs, files in os.walk(base_dir):
        if files:
            parts = root.split(os.sep)
            
            # Ensure the path has enough components
            if len(parts) >= 7:
                _, year, before_exp, state, stim_type, hemisphere, subject = parts[-7:]
                
                # Build new path
                new_study = f"Stim-{stim_type}_exp"
                new_base = os.path.join(base_dir, new_study, subject)
                new_folder = os.path.join(new_base, state, f"{hemisphere}-{stim_type}")
                
                os.makedirs(new_folder, exist_ok=True)

                # Move files
                for file in files:
                    old_path = os.path.join(root, file)
                    new_path = os.path.join(new_folder, file)
                    shutil.move(old_path, new_path)

                print(f"Moved: {subject} → {new_folder}")

    print("Done!")

In [ ]:
restructure_experiment_folders("D:\\Stimulation_2024\\Before_exp")

# Example function for renaming experiments

In [7]:
from pathlib import Path

def rename_protocol_folders(base_dir):
    """
    Replaces underscores with dashes in protocol-level folder names:
    base_dir/<mouse>/<state>/<protocol> becomes base_dir/<mouse>/<state>/<protocol-with-dashes>

    Parameters
    ----------
    base_dir : str or Path
        Root directory containing the mouse/state/protocol hierarchy.

    Returns
    -------
    int
        Number of folders renamed.
    """
    base_path = Path(base_dir)
    rename_count = 0

    for mouse_dir in base_path.iterdir():
        if not mouse_dir.is_dir():
            continue
        for state_dir in mouse_dir.iterdir():
            if not state_dir.is_dir():
                continue
            for protocol_dir in list(state_dir.iterdir()):  # list() to avoid runtime rename issues
                if not protocol_dir.is_dir():
                    continue
                old_name = protocol_dir.name
                if "_" in old_name:
                    new_name = old_name.replace("_", "-")
                    new_path = protocol_dir.parent / new_name
                    if not new_path.exists():  # avoid overwriting
                        protocol_dir.rename(new_path)
                        print(f"Renamed: {protocol_dir} -> {new_path}")
                        rename_count += 1
                    else:
                        print(f"Skip (already exists): {new_path}")
    return rename_count

In [8]:
renamed = rename_protocol_folders(r"E:\PT_2023\Before_PT")
print(f"Total folders renamed: {renamed}")

Renamed: E:\PT_2023\Before_PT\m10j\Awake\LHL_60mkA -> E:\PT_2023\Before_PT\m10j\Awake\LHL-60mkA
Renamed: E:\PT_2023\Before_PT\m10j\Awake\LHL_90mkA -> E:\PT_2023\Before_PT\m10j\Awake\LHL-90mkA
Renamed: E:\PT_2023\Before_PT\m10j\Awake\RHL_60mkA -> E:\PT_2023\Before_PT\m10j\Awake\RHL-60mkA
Renamed: E:\PT_2023\Before_PT\m10j\Awake\RHL_90mkA -> E:\PT_2023\Before_PT\m10j\Awake\RHL-90mkA
Renamed: E:\PT_2023\Before_PT\m10j\Iso\LHL_60mkA -> E:\PT_2023\Before_PT\m10j\Iso\LHL-60mkA
Renamed: E:\PT_2023\Before_PT\m10j\Iso\LHL_90mkA -> E:\PT_2023\Before_PT\m10j\Iso\LHL-90mkA
Renamed: E:\PT_2023\Before_PT\m10j\Iso\RHL_60mkA -> E:\PT_2023\Before_PT\m10j\Iso\RHL-60mkA
Renamed: E:\PT_2023\Before_PT\m10j\Iso\RHL_90mkA -> E:\PT_2023\Before_PT\m10j\Iso\RHL-90mkA
Renamed: E:\PT_2023\Before_PT\m11j\Awake\LHL_60mkA -> E:\PT_2023\Before_PT\m11j\Awake\LHL-60mkA
Renamed: E:\PT_2023\Before_PT\m11j\Awake\LHL_90mkA -> E:\PT_2023\Before_PT\m11j\Awake\LHL-90mkA
Renamed: E:\PT_2023\Before_PT\m11j\Awake\RHL_60mkA -> E:

# Example function for listing mouse, state, protocol, and experiment names

In [7]:
from pathlib import Path
from typing import List, Tuple, Union

def extract_unique_exp_mouse_state_protocol(
    base_dir: Union[str, Path]
) -> Tuple[List[str], List[str], List[str], List[str], List[Tuple[str, str, str, str]]]:
    """Extracts unique experiment structure elements from nested directory layout.

    Traverses a directory structure of the form:
    base_dir/<exp>/<mouse>/<state>/<protocol>, and collects unique values
    for each level (experiment, mouse, state, protocol). Also compiles a full list
    of experiment combinations.

    Args:
        base_dir (Union[str, Path]): Root directory containing experiment folders.

    Returns:
        Tuple[List[str], List[str], List[str], List[str], List[Tuple[str, str, str, str]]]:
            - List of unique experiment names (exp).
            - List of unique mouse identifiers.
            - List of unique state names.
            - List of unique protocol names.
            - List of tuples representing each full (exp, mouse, state, protocol) path.
    """
    base_path = Path(base_dir)
    exps = set()
    mice = set()
    states = set()
    protocols = set()

    for exp_dir in base_path.iterdir():
        if not exp_dir.is_dir():
            continue
        exps.add(exp_dir.name)

        for mouse_dir in exp_dir.iterdir():
            if not mouse_dir.is_dir():
                continue
            mice.add(mouse_dir.name)

            for state_dir in mouse_dir.iterdir():
                if not state_dir.is_dir():
                    continue
                states.add(state_dir.name)

                for protocol_dir in state_dir.iterdir():
                    if protocol_dir.is_dir():
                        protocols.add(protocol_dir.name)

    return (
        sorted(exps),
        sorted(mice),
        sorted(states),
        sorted(protocols)
    )

In [9]:
base_dir = r"D:\Test_exp"
exp, mice, states, protocols = extract_unique_exp_mouse_state_protocol(base_dir)

print("Experiments:", exp)
print("Mice:", mice)
print("States:", states)
print("Protocols:", protocols)

Experiments: ['Before-exp']
Mice: ['m4433', 'm8j']
States: ['Awake', 'Iso']
Protocols: ['LHL-90mkA', 'RE-1-5', 'RHL-90mkA', 'RV-pulse09', 'Rest-state']
